In [ ]:
"""df_ff=df_0.copy()
df_ff.loc[(df_ff["PROVINCIA_RESIDENCIA"]=="Buenos Aires")&(df_ff["LOCALIDAD_RESIDENCIA"]=="San Martin"),"LOCALIDAD_RESIDENCIA"]="General Jose de San Martin"
dic_cambios = {"1Deg de Mayo" : "1 de Mayo",
"Libertador General San Martin":"General Jose de San Martin",
'Libertador Grl. San Martin':"General Jose de San Martin",    
"Grl. Jose de San Martin":"General Jose de San Martin",
"Gral Manuel Belgrano":"Dr. Manuel Belgrano",
"General San Martin":"General Jose de San Martin",
"Libertador General Jose de San Martin":"General Jose de San Martin",
"Santa Victoria Oeste":"Santa Victoria",
"Lib. Gral. San Martin":"Libertador General San Martin",
"Libertador General San Martin":"General Jose de San Martin",
"Juan Martin de Pueyrredon":"La Capital",
"General M. Belgrano":"General Manuel Belgrano",
"Lib. Gral. San Martin":"General Jose de San Martin",
"Leandro N.Alem":"Leandro N. Alem",
"O Higgins":"O'Higgins",
"Fray Justo S.M. de Oro":"Fray Justo Santa Maria de Oro",
"Rosario V. Penaloza":"Rosario Vera Penaloza",

"Pte. Roque Saenz Pena":"Presidente Roque Saenz Pena",
"Adolfo Gonzalez Chaves":"Adolfo Gonzales Chaves",
"Coronel L. Rosales":"Coronel de Marina Leonardo Rosales",
"Coronel de Marina L. Rosales":"Coronel de Marina Leonardo Rosales",
"Jose C Paz":"Jose C. Paz",

"General Sarmiento":"Capitan Sarmiento",

"General Lamadrid":"General la Madrid",
"Eldorado":"El Dorado",
"Juan B. Alberdi":"Alberdi",
"1Deg de Mayo":"1 de Mayo",
"Islas de Ibicuy":"Islas del Ibicuy",
"1O de Mayo":"1 de Mayo",
"Eldorado":"El Dorado",
"Libertador General San Martin":"General Jose de San Martin",
"General San Martin":"General Jose de San Martin",
"General Lamadrid":"General la Madrid",
"Juan B. Alberdi":"Alberdi",
"10 de Mayo":"1 de Mayo"
}



for s in dic_cambios:
    df_ff["DEPARTAMENTO_RESIDENCIA"]=df_ff["DEPARTAMENTO_RESIDENCIA"].str.replace(s,dic_cambios[s])
    df_provincias['Departamento']=df_provincias['Departamento'].str.replace(s,dic_cambios[s])
    #df_casos_activos_depto_fecha["DEPARTAMENTO_RESIDENCIA"]=df_casos_activos_depto_fecha["DEPARTAMENTO_RESIDENCIA"].str.replace(s,dic_cambios[s])

dic_cambios={
"Esteban Echeverria":"Buenos Aires",
"Florencio Varela":"Buenos Aires",
"San Fernando":"Buenos Aires",
"General Jose de San Martin":"Buenos Aires",
}

for s in dic_cambios:
    df_ff.loc[(df_ff["PROVINCIA_RESIDENCIA"]=="Desconocida")&(df_ff["DEPARTAMENTO_RESIDENCIA"]==s),"PROVINCIA_RESIDENCIA"]=dic_cambios[s]
    
    
df_ff.loc[df_ff["DEPARTAMENTO_RESIDENCIA"].astype(str).str.contains("Comuna"),"PROVINCIA_RESIDENCIA"]="Caba"
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cambios_segun_loc(df_f,df_provincias):


    df_f["Arreglos_Localidades"]=0
    i=0

    df_f["PROVINCIA_RESIDENCIA_2"]=df_f["PROVINCIA_RESIDENCIA"]
    df_f["DEPARTAMENTO_RESIDENCIA_2"]=df_f["DEPARTAMENTO_RESIDENCIA"]
    df_f["LOCALIDAD_RESIDENCIA_2"]=df_f["LOCALIDAD_RESIDENCIA"]

    for p in df_f["PROVINCIA_RESIDENCIA"].unique():
        bool_prov=df_f["PROVINCIA_RESIDENCIA"]==p

        b_prov=(df_provincias["Provincia"]==p)
        
        for d in df_f.loc[bool_prov,"DEPARTAMENTO_RESIDENCIA"].unique():

            b_dep=(df_provincias["Departamento"]==d)
            deptos_limpios = [elemento for elemento in list(df_provincias.loc[b_dep,"Departamento"].unique())]


            
            bool_dep=df_f["DEPARTAMENTO_RESIDENCIA"]==d
            
            for l in df_f.loc[bool_prov&bool_dep,"LOCALIDAD_RESIDENCIA"].unique():

                entidades_limpias = [elemento for elemento in list(df_provincias.loc[b_dep&b_prov,"Entidad comprendida"].unique())] #Para comparar y ver si cuadra con Provincia y Depto
                entidades_otras = [elemento  for elemento in list(df_provincias["Entidad comprendida"].unique())]

                entidades_deptos = [elemento for elemento in list(df_provincias.loc[b_dep,"Entidad comprendida"].unique())]          
                entidades_provincias = [elemento for elemento in list(df_provincias.loc[b_dep,"Entidad comprendida"].unique())] 
                if l not in entidades_limpias and l!="Nan" and d!="Nan":
                    print("Incorrecto",p,d,l)
                    bool_loc=df_f["LOCALIDAD_RESIDENCIA"]==l
                    index_incorrecto=df_f.loc[(df_f["PROVINCIA_RESIDENCIA"]==p)&
                                            (df_f["DEPARTAMENTO_RESIDENCIA"]==d)&
                                            (df_f["LOCALIDAD_RESIDENCIA"]==l)].index

                    #if l in entidades_otras or l in localidades_otras: 

                    if l in entidades_otras: 
                        
                        
                        #if l in entidades_otras:
                        df_correcto=(df_provincias.loc[df_provincias["Entidad comprendida"]==l]).iloc[0]


                            
                        prov,depto,local=df_correcto["Provincia"],df_correcto["Departamento"],df_correcto["Entidad comprendida"]
                        print("Correcto",prov,depto,local)
                        print()
                        #display(df_f.loc[index_incorrecto][["PROVINCIA_RESIDENCIA","DEPARTAMENTO_RESIDENCIA","LOCALIDAD_RESIDENCIA"]])
                        df_f.loc[index_incorrecto,"PROVINCIA_RESIDENCIA_2"]=prov
                        df_f.loc[index_incorrecto,"DEPARTAMENTO_RESIDENCIA_2"]=depto
                        df_f.loc[index_incorrecto,"LOCALIDAD_RESIDENCIA_2"]=local
                        df_f.loc[index_incorrecto,"Arreglos_Localidades"]=1
                        #display(df_f.loc[index_incorrecto][["PROVINCIA_RESIDENCIA","DEPARTAMENTO_RESIDENCIA","LOCALIDAD_RESIDENCIA"]])                 

                    else:
                        corte=1.5
                        maximo=1.8
                        if maximo<1:
                            for ll in entidades_otras:
                

                                # Define las dos cadenas que deseas comparar


                                # Crea un objeto CountVectorizer para convertir las cadenas en vectores numéricos
                                vectorizer = CountVectorizer().fit_transform([l, ll])

                                # Calcula la similaridad de coseno entre los vectores
                                similarity = cosine_similarity(vectorizer)

                                # Imprime la similaridad de coseno
                                if similarity[0,1]>maximo:
                                    maximo=similarity[0,1]
                                    mejor_localidad=ll
                            if maximo>0.8:
                                df_correcto=(df_provincias.loc[df_provincias["Entidad comprendida"]==mejor_localidad]).iloc[0]
                                prov,depto,local=df_correcto["Provincia"],df_correcto["Departamento"],df_correcto["Entidad comprendida"]
                                df_f.loc[index_incorrecto,"PROVINCIA_RESIDENCIA_2"]=prov
                                df_f.loc[index_incorrecto,"DEPARTAMENTO_RESIDENCIA_2"]=depto
                                df_f.loc[index_incorrecto,"LOCALIDAD_RESIDENCIA_2"]=local
                                df_f.loc[index_incorrecto,"Arreglos_Localidades"]=2
                                print("Correcto2",prov,depto,local)
                                print("Similaridad de",maximo) 
                                print()


                            i+=1   

    return(df_f)

df_f=cambios_segun_loc(df_ff,df_provincias)
df_f["Arreglos_Localidades"].sum() # se ve la cantidad de arreglos
"""
